In [1]:
import warnings
warnings.filterwarnings('ignore')

import json
import numpy as np
import pandas as pd

from joblib import Parallel, delayed  
import multiprocessing
num_cores = multiprocessing.cpu_count()





In [2]:
def getMovieData(name):
    
    import json
    import boxofficemojoAPI as bom
   
    
    bo = bom.BoxOfficeMojo()
    bo.crawl_for_urls()
    data1 = bo.get_movie_summary(name)
    data1.clean_data()
    data1 = json.loads(data1.to_json())
    
    data2 = bo.get_weekly_summary(name)
    data2.clean_data()
    data2 = json.loads(data2.to_json())['weekly']
    
    Features = dict()
    try:
        Features['director'] = data1["directors"]
    except KeyError:
        Features['director'] = None
        
    try:
        Features['distributor'] = data1["distributor"]
    except KeyError:
        Features['distributor'] = None
        
    try:
        Features['domestic_BO'] = data1["domestic"]
    except KeyError:
        Features['domestic_BO'] = None
    try:
        Features['foreign_BO'] = data1["foreign"]
    except KeyError:
        Features['foreign_BO'] = None
    
    try:
        Features['genre'] = data1["genre"]
    except KeyError:
        Features['genre'] = None
        
    try:
        Features['rating'] = data1["mpaa_rating"]
    except KeyError:
         Features['rating'] = None
    
    try:
        Features['budget'] = data1["production_budget"]
    except KeyError:
        Features['budget'] = None
        
    try:
        Features['runtime'] = data1["runtime"]
    except KeyError:
        Features['runtime'] = None
        
    try:
        Features['writers'] = data1["writers"]
    except KeyError:
        Features['writers'] = None
        
    
    starting_week = data2[0]['week_number']
    if starting_week == 0:
        del data2[0]
        
    del starting_week

    for k in range(min(len(data2), 15)):
        if k == 0:
            
            try:
                Features['_'.join(('week', str(k+1),'gross'))] = data2[k]['gross']
            except KeyError:
                Features['_'.join(('week', str(k+1),'gross'))] = None
                
            try:
                Features['_'.join(('week', str(k+1),'rank'))] = data2[k]['rank']
            except KeyError:
                Features['_'.join(('week', str(k+1),'rank'))] = None
                
            try:
                Features['_'.join(('week', str(k+1),'avg'))] = data2[k]['average_per_theatre']
            except KeyError:
                Features['_'.join(('week', str(k+1),'avg'))] = None
                
        else :

            Features['_'.join(('week', str(k+1), 'change'))]= data2[k]['week_over_week_change']
            Features['_'.join(('week', str(k+1), 'rank'))]= data2[k]['rank']
            Features['_'.join(('week', str(k+1),'avg'))] = data2[k]['average_per_theatre']
            
    return Features
            





In [3]:
Marvel =[
    'amazingspiderman2', 
    'antman', 
    'avengers11','avengers2', 
    'blade','blade2', 'blade3', 
    'captainamerica', 
    'daredevil', 
    'deadpool2016',
    'disney2014', 
    'elektra', 
    'fantasticfour', 'fantasticfour15','fantasticfour2', 
    'ghostrider', 'ghostrider2', 
    'howardtheduck',
    'hulk', 
    'incrediblehulk', 
    'ironman', 'ironman2', 'ironman3',
    'marvel14b', 'marvel2014a', 'marvel2016', 
    'punisher', 'punisher2',
    'spiderman', 'spiderman2', 'spiderman3', 'spiderman4',
    'thor','thor2', 
    'wolverine', 'wolverine2', 
    'x-men2016', 'x2', 'x3', 'xmen','xmen2014', 'xmenfirstclass'
]



Marvel_titles =[
    "The Amazing Spider-Man 2",
    "Ant-Man",
    "Marvel's The Avengers","Avengers: Age of Ultron",
    "Blade","Blade II","Blade: Trinity",
    "Captain America: The First Avenger",
    "Daredevil",
    "Deadpool",
    'Big Hero 6',
    "Elektra",
    "Fantastic Four(2005)","Fantastic Four(2015)","Fantastic Four: Rise of the Silver Surfer",
    "Ghost Rider","Ghost Rider: Spirit of Vengeance",
    "Howard The Duck", 
    "Hulk","The Incredible Hulk",
    "Iron Man","Iron Man 2","Iron Man 3",
    "Captain America: The Winter Soldier",
    "Guardians of the Galaxy",
    "Captain America: Civil War",
    "The Punisher","Punisher: War Zone",
    "Spider-Man","Spider-Man 2","Spider-Man 3","The Amazing Spider-Man",
    "Thor","Thor: The Dark World",
    "X-Men Origins: Wolverine","The Wolverine",
    "X-Men:Apocalypse","X2: X-Men United", "X-Men: The Last Stand",
    "X-Men","X-Men: Days of Future Past","X-Men: First Class"
]



In [4]:
len(Marvel_titles)

42

In [4]:


Dc = [ 'batman', 'batman3', 'batmanbegins', 'batmanforever',
       'batmanmaskofthephantasm', 'batmanreturns', 'batmanrobin',
       'catwoman', 
       'constantine',  
       'darkknight', 
       'dc2016',
       'greenlantern', 
       'jonahhex',
       'losers', 
       'red2', 'red2010', 
       'steel',
       'supergirl', 
       'superman','superman06', 'superman2', 'superman2012', 'superman2015',
       'superman3', 'superman4', 'vforvendetta', 'watchmen']


Dc_titles =[
    "Batman","The Dark Knight Rises","Batman Begins","Batman Forever",
    "Batman: Mask of the Phantasm","Batman Returns","Batman Robin", 
    "Catwoman",
    "Constantine",
    "The Dark Knight",
    "Suicide Squad",
    "Green Lantern",
    "Jonah Hex",
    "The Losers",
    "Red 2","Red",
    "Steel",
    "Supergirl",
    "Superman","Superman Returns", "Superman II","Man of Steel","Batman v Superman: Dawn of Justice",
    "Superman III","Superman IV: The Quest for Peace", 
    "V for Vendetta", "Watchmen"
]

In [5]:
Others_titles = [
    "2 Guns","300","300: Rise of an Empire","30 Days of Night",
    "American Splendor","Alien vs Predator","Alien vs Predator Requiem",
    "Black Mask","Buffy the Vampire Slayer",
    "Bulletproof Monk","Bullet to the Head",
    "Casper","Chronicle (2012)",
    "Cowboys & Aliens",
    "The Crow",
    "The Crow:City of Angels",
    "Darkman",
    "The Diary of a Teenage Girl", "Dick Tracy",
    "Dredd",
    "From Hell",
    "Ghost world",
    "The Green Hornet",
    "Hancock",
    "Hellboy",
    "Hellboy II: The Golden Army",
    "Hercules",
    "A history of violence",
    "I, Frankenstein",
    "The Incredibles",
    "Iron Monkey",
    "Josie and the Pussycats",
    "Judge Dredd",
    "Kick-Ass",
    "Kick-Ass 2",
    "The League of Extraordinary Gentlemen",
    "The Mask",
    "The Mask of Zorro",
    "Megamind",
    "Men in Black",
    "Men in Black II",
    "Men in Black III",
    "Monkeybone",
    "Mystery Men",
    "Oblivion",
    "The Phantom",
    "The Powerpuff Girls Movie",
    "Mighty Morphin' Power Rangers",
    "Priest",
    "Richie Rich",    
    "RIPD",
    "Road to Perdition",
    "The Rocketeer",
    "Scott Pilgrim vs the World",
    "Kingsman: The Secret Service",
    "The Shadow",
    "Sin City",
    "Sin City: A Dame to Kill For",
    "Sky High",
    "Snowpiercer","Spawn","The Spirit",
    "Super",
    "My Super Ex-Girlfriend",
    "Superhero Movie",
    "Surrogates",
    "Tank Girl ",
    "Teenage Mutant Ninja Turtles",
    "Timecop",
    "The Adventures of Tintin",
    "Teenage Mutant Ninja Turtles (2014)",
    "Teenage Mutant Ninja Turtles: Out of the Shadow",
    "Teenage Mutant Ninja Turtles III","TMNT",
    "Teenage Mutant Ninja Turtles II: The Secret of the Ooze",
    "Virus",
    "Wanted",
    "Whiteout",
    "The X-Files: I Want to Believe",
    "Zoom",
    "The Legend of Zorro"

]

Others = [
       '2guns', '300', '300sequel', '30daysofnight', 'americansplendor',
       'avp', 'avp2', 'blackmask', 'buffythevampireslayer',
       'bulletproofmonk', 'bullettothehead', 'casper', 'chronicle2012',
       'cowboysandaliens', 'crow', 'crowcityofangels', 
       'darkman',
       'diaryofateenagegirl', 'dicktracy', 'dredd', 'fromhell',
       'ghostworld', 'greenhornet', 'hancock', 'hellboy', 'hellboy2',
       'hercules2014', 'historyofviolence', 'ifrankenstein', 'incredibles',
       'ironmonkey', 'josieandthepussycats', 'judgedredd', 'kickass',
       'kickass2', 'leagueofextraordinarygentlemen', 'mask', 'maskofzorro',
       'mastermind', 'meninblack', 'meninblack2', 'mib3', 'monkeybone',
       'mysterymen', 'oblivion', 'phantom', 'powerpuffgirls',
       'powerrangers', 'priest07', 'richierich', 'ripd', 'roadtoperdition',
       'rocketeer', 'scottpilgrim', 'secretservice', 'shadow', 'sincity',
       'sincity2', 'skyhigh', 'snowpiercer', 'spawn', 'spirit09',
       'super2011', 'superexgirlfriend', 'superheromovie', 'surrogates',
       'tankgirl', 'teenagemutantninjaturtles', 'timecop', 'tintin',
       'tmnt2013', 'tmnt2016', 'tmnt3', 'tmntcg', 'tnmt2', 'virus',
       'wanted', 'whiteout', 'xfiles2', 'zoom', 'zorro2'
]


In [6]:
%%capture



# MARVEL = Parallel(n_jobs=num_cores)(delayed(getMovieData)(name) for name in Marvel)
# DC = Parallel(n_jobs=num_cores)(delayed(getMovieData)(name) for name in Dc)
# OTHERS = Parallel(n_jobs=num_cores)(delayed(getMovieData)(name) for name in Others)


In [7]:
len(Others_titles)

81

In [8]:
len(OTHERS)

81

In [9]:
# table = pd.DataFrame(MARVEL)
# table['Title'] = np.array(Marvel_titles)
# table.to_csv('../R/marvel_bom.csv',na_rep='', header= True, index= False, index_label= False)



# table = pd.DataFrame(DC)
# table['Title'] = np.array(Dc_titles)
# table.to_csv('../R/dc_bom.csv',na_rep='', header= True, index= False, index_label= False)




# table = pd.DataFrame(OTHERS)
# table['Title'] = np.array(Others_titles)
# table.to_csv('../R/others_bom.csv',na_rep='', header= True, index= False, index_label= False)

In [10]:
table

,budget,director,distributor,domestic_BO,foreign_BO,genre,rating,runtime,week_10_avg,week_10_change,...,week_7_change,week_7_rank,week_8_avg,week_8_change,week_8_rank,week_9_avg,week_9_change,week_9_rank,writers,Title
0,61000000.0,[Baltasar Kormakur],Universal,75612460.0,56327951.0,Action Comedy,R,109,945.0,-0.529,...,-0.351,15.0,1195.0,-0.275,17.0,995.0,-0.554,22.0,[Blake Masters],2 Guns
1,65000000.0,[Zack Snyder],Warner Bros.,210614939.0,245453242.0,Period Action,R,117,1673.0,-0.371,...,-0.470,11.0,1932.0,-0.377,15.0,1414.0,-0.685,18.0,"[Michael B. Gordon, Kurt Johnstad, Frank Mille...",300
2,110000000.0,[Noam Murro],Warner Bros.,106580051.0,231000000.0,Period Action,R,102,604.0,-0.346,...,-0.503,25.0,1022.0,-0.004,24.0,750.0,-0.469,31.0,"[Kurt Johnstad, Zack Snyder]",300: Rise of an Empire
3,30000000.0,[David Slade],Sony / Columbia,39568996.0,35936977.0,Horror,R,113,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Stuart Beattie],30 Days of Night
4,NaN,None,Fine Line,6010990.0,1975094.0,Comedy,R,101,978.0,-0.439,...,-0.431,24.0,1511.0,-0.283,28.0,1338.0,-0.464,38.0,None,American Splendor
5,60000000.0,[Paul W.S. Anderson],Fox,80282231.0,92262423.0,Sci-Fi Horror,PG-13,87,893.0,0.027,...,-0.472,33.0,725.0,-0.343,35.0,854.0,-0.061,35.0,[Paul W.S. Anderson],Alien vs Predator
6,NaN,"[Colin Strause, Greg Strause]",Fox,41797066.0,87087428.0,Sci-Fi Horror,R,86,513.0,-0.346,...,-0.052,51.0,663.0,-0.508,61.0,588.0,-0.533,71.0,[Shane Salerno],Alien vs Predator Requiem
7,NaN,None,Artisan,12504289.0,NaN,Action,R,102,NaN,NaN,...,NaN,38.0,697.0,-0.143,40.0,565.0,-0.590,57.0,None,Black Mask
8,NaN,None,Fox,16624456.0,NaN,Horror Comedy,PG-13,86,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Joss Whedon],Buffy the Vampire Slayer
9,52000000.0,None,MGM,23358708.0,14355171.0,Action,PG-13,103,669.0,-0.237,...,-0.115,48.0,782.0,0.037,44.0,733.0,0.003,46.0,None,Bulletproof Monk
